## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-31-20-48-00 +0000,wsj,The Famous Names in the Latest Release of Epst...,https://www.wsj.com/us-news/law/what-we-know-s...
1,2026-01-31-20-44-21 +0000,nyt,Federal Judge Denies Request to Block ICE Surg...,https://www.nytimes.com/2026/01/31/us/judge-mi...
2,2026-01-31-20-43-05 +0000,nyt,Large Lizard Is Rescued After It’s Found Burie...,https://www.nytimes.com/2026/01/31/us/lizard-r...
3,2026-01-31-20-39-13 +0000,bbc,Coast Guard suspends search for missing crew a...,https://www.bbc.com/news/articles/cvgjxxknj3do...
4,2026-01-31-20-39-10 +0000,nyt,Escape to Alcatraz: Coyote Swims a Mile in Sea...,https://www.nytimes.com/2026/01/30/us/coyote-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2480/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
159,trump,33
10,ice,26
4,epstein,23
36,new,11
119,fed,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
124,2026-01-31-10-30-00 +0000,wsj,Trump’s pick for Fed chair has long attacked t...,https://www.wsj.com/economy/central-banking/ke...,85
180,2026-01-31-02-00-00 +0000,wsj,"Trump’s fed pick came down to the “two Kevins,...",https://www.wsj.com/politics/policy/how-warsh-...,71
231,2026-01-30-22-10-00 +0000,latimes,Granderson: Even as Trump shreds the Constitut...,https://www.latimes.com/opinion/story/2026-01-...,70
114,2026-01-31-11-00-00 +0000,latimes,Trump officials have tried to justify ICE shoo...,https://www.latimes.com/california/story/2026-...,64
82,2026-01-31-15-16-54 +0000,bbc,Former CNN host Don Lemon charged in anti-ICE ...,https://www.bbc.com/news/articles/c24gm3dz36po...,63


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
124,85,2026-01-31-10-30-00 +0000,wsj,Trump’s pick for Fed chair has long attacked t...,https://www.wsj.com/economy/central-banking/ke...
82,63,2026-01-31-15-16-54 +0000,bbc,Former CNN host Don Lemon charged in anti-ICE ...,https://www.bbc.com/news/articles/c24gm3dz36po...
34,60,2026-01-31-18-50-44 +0000,nyt,"New Epstein Files Name Elon Musk, Bill Gates a...",https://www.nytimes.com/2026/01/31/us/epstein-...
146,56,2026-01-31-05-07-00 +0000,wsj,The Senate passed a bipartisan deal Friday to ...,https://www.wsj.com/politics/policy/senate-set...
190,35,2026-01-31-00-50-39 +0000,bbc,I mocked the Saudi leader on YouTube - then my...,https://www.bbc.com/news/articles/cj6w3zgden0o...
28,33,2026-01-31-19-21-58 +0000,nypost,"TV star fisherman, crew missing off Massachuse...",https://nypost.com/2026/01/31/us-news/tv-star-...
179,31,2026-01-31-02-00-00 +0000,wsj,Shifting views on the president’s immigration ...,https://www.wsj.com/politics/elections/the-tru...
39,29,2026-01-31-18-35-37 +0000,nypost,"US, Israel deny involvement in mysterious blas...",https://nypost.com/2026/01/31/world-news/us-is...
169,28,2026-01-31-02-31-23 +0000,latimes,Ex-Google engineer convicted of stealing AI tr...,https://www.latimes.com/california/story/2026-...
14,27,2026-01-31-20-08-42 +0000,nypost,LA’s top fed prosecutor Bill Essayli deploys m...,https://nypost.com/2026/01/31/us-news/bill-ess...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
